In [ ]:
# Implementation of word2vec on the IMDB 5000 dataset

In [1]:
from __future__ import absolute_import, division, print_function
import multiprocessing, os, pprint, re
import nltk
import gensim.models.word2vec as word2vec
import sklearn.manifold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re, string

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('movie_metadata.csv')

In [3]:
df = df.drop(df.movie_imdb_link.name, 1)
def trim_and_remove_spaces(raw):
    if pd.isnull(raw) == False:
        return raw.strip().replace(' ', '_')

df.director_name = df.director_name.apply(trim_and_remove_spaces)
df.actor_2_name = df.actor_2_name.apply(trim_and_remove_spaces)

df.actor_1_name = df.actor_1_name.apply(trim_and_remove_spaces)
df.movie_title = df.movie_title.apply(trim_and_remove_spaces)
df.actor_3_name = df.actor_3_name.apply(trim_and_remove_spaces)
df.language = df.language.apply(trim_and_remove_spaces)
df.country = df.country.apply(trim_and_remove_spaces)

In [4]:
def descretize3(df, col):
    
    if(df[col].dtype == np.float64 or df[col].dtype == np.int64):
        df[col] = pd.qcut(df[col], 3, labels=['low', 'medium', 'high'], retbins=False)
        df[col] = df[col].apply(lambda x : (str(x)).replace(" ", ""))
        
def descretize2(df, col):
    
    if(df[col].dtype == np.float64 or df[col].dtype == np.int64):
        df[col] = pd.qcut(df[col], 2, labels=['low', 'high'], retbins=False)
        df[col] = df[col].apply(lambda x : (str(x)).replace(" ", ""))
        
def descretize10(df, col):
    
    if(df[col].dtype == np.float64 or df[col].dtype == np.int64):
        df[col] = pd.qcut(df[col], 3, labels=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], retbins=False)
        df[col] = df[col].apply(lambda x : (str(x)).replace(" ", ""))

descretize3(df, df.director_facebook_likes.name)
descretize3(df, df.num_critic_for_reviews.name)
descretize3(df, df.duration.name)
descretize3(df, df.actor_3_facebook_likes.name)
descretize3(df, df.actor_1_facebook_likes.name)

descretize3(df, df.gross.name)
descretize3(df, df.num_voted_users.name)
descretize3(df, df.cast_total_facebook_likes.name)
descretize3(df, df.num_user_for_reviews.name)
descretize3(df, df.budget.name)

descretize3(df, df.actor_2_facebook_likes.name)
descretize3(df, df.imdb_score.name)
descretize2(df, df.facenumber_in_poster.name)
descretize3(df, df.title_year.name)
descretize3(df, df.aspect_ratio.name)
descretize2(df, df.movie_facebook_likes.name)

In [5]:
from sklearn.utils import shuffle
df = shuffle(df)
df

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
1771,Color,Scott_Hicks,high,medium,medium,high,Lily_Rabe,medium,medium,Comedy|Drama|Romance,Zoë_Kravitz,No_Reservations,medium,medium,Bob_Balaban,high,chef|cooking|guardian|kitchen|restaurant,medium,English,USA,PG,medium,medium,medium,medium,medium,low
2114,Color,Blake_Edwards,low,high,high,low,Vernon_Dobtcheff,low,low,Comedy|Drama|Musical|Romance|War,Rock_Hudson,Darling_Lili,low,low,Jeremy_Kemp,high,crepe suzette|french|mata hari spoof|song|spy,low,English,USA,G,medium,low,low,medium,medium,low
2545,Black and White,Michel_Hazanavicius,high,medium,high,high,Ed_Lauter,medium,medium,Comedy|Drama|Romance,Bérénice_Bejo,The_Artist,high,medium,Beth_Grant,low,1920s|jack russell terrier|modern silent movie...,high,English,France,PG-13,medium,high,high,high,low,high
4737,Color,Greg_Harrison,low,low,low,low,Ari_Gold,low,low,Drama|Music,Rachel_True,Groove,low,low,Denny_Kirkwood,low,dj|ecstasy the drug|rave|raver|warehouse,low,English,USA,R,low,low,low,medium,low,low
1770,Color,Kenny_Ortega,low,low,high,low,Omri_Katz,low,medium,Comedy|Family|Fantasy,Vinessa_Shaw,Hocus_Pocus,medium,low,Larry_Bagby,low,candle|cat|halloween|spell|virgin,medium,English,USA,PG,medium,low,low,medium,low,high
4568,Color,Don_Coscarelli,high,low,high,low,Ossie_Davis,medium,NaN,Comedy|Fantasy|Mystery,Daniel_Roebuck,Bubba_Ho-Tep,medium,low,Ella_Joyce,high,cult film|elvis|elvis presley|nursing home|texas,high,English,USA,R,low,medium,low,high,low,low
4414,Color,Alex_Cox,medium,low,medium,low,Tracey_Walter,low,NaN,Comedy|Crime|Sci-Fi|Thriller,Del_Zamora,Repo_Man,medium,low,Miguel_Sandoval,high,punk|punk rocker|repossession|televangelist|ufo,medium,English,USA,R,low,low,low,medium,low,low
602,Color,Tim_Burton,high,high,high,high,Alison_Lohman,high,high,Adventure|Drama|Fantasy,Steve_Buscemi,Big_Fish,high,high,Albert_Finney,low,dog saved from a fire|flashback|growing up|mul...,high,English,USA,PG-13,high,medium,high,high,low,high
1773,Color,Ruben_Fleischer,high,low,high,medium,Dilshad_Vadsaria,low,medium,Action|Comedy|Crime,Bianca_Kajlich,30_Minutes_or_Less,high,medium,Fred_Ward,high,bank heist|bank robbery|heist gone wrong|pizza...,medium,English,Germany,R,medium,high,medium,low,medium,low
1518,Color,George_Tillman_Jr.,medium,high,medium,high,Melissa_Benoist,medium,medium,Drama|Romance,Tiago_Riani,The_Longest_Ride,medium,medium,Hayley_Lovitt,low,1940s|art collector|bull riding|north carolina...,medium,English,USA,PG-13,high,high,high,high,medium,high


In [6]:
raw_sentences = []
for a in df.iterrows():
    words = []    
    for b in a[1].iteritems():
        if b[0] == 'genres':
            for g in str(b[1]).split("|"):
                words.append('genre')
                words.append(trim_and_remove_spaces(g.lower()))
        elif b[0] == 'plot_keywords':
            for p in str(b[1]).split("|"):
                words.append('plot')
                words.append(trim_and_remove_spaces(p.lower()))
        elif b[0] in ['director_name', 'actor_2_name', 'actor_1_name', 'actor_3_name']:
            words.append(str(b[0]))
            words.append(str(b[1]))
        else:
            words.append(str(b[0]) + "_" + str(b[1]))
    raw_sentences.append(' '.join(words))

In [7]:
def sentence_to_wordlist(raw):
    words = raw.split()
    return words

tokenized_sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        tokenized_sentences.append(sentence_to_wordlist(raw_sentence))

In [8]:
token_count = sum([len(sentence) for sentence in tokenized_sentences])
print('Total token count = {0:,}'.format(token_count))

Total token count = 224,498


In [9]:
tokenized_sentences[51]

['color_Color',
 'director_name',
 'Roland_Emmerich',
 'num_critic_for_reviews_high',
 'duration_high',
 'director_facebook_likes_high',
 'actor_3_facebook_likes_medium',
 'actor_2_name',
 'Sela_Ward',
 'actor_1_facebook_likes_medium',
 'gross_high',
 'genre',
 'action',
 'genre',
 'adventure',
 'genre',
 'sci-fi',
 'actor_1_name',
 'Vivica_A._Fox',
 'movie_title_Independence_Day:_Resurgence',
 'num_voted_users_medium',
 'cast_total_facebook_likes_medium',
 'actor_3_name',
 'Judd_Hirsch',
 'facenumber_in_poster_low',
 'plot',
 'alien',
 'plot',
 'battle',
 'plot',
 'defense',
 'plot',
 'independence_day',
 'plot',
 'mothership',
 'num_user_for_reviews_high',
 'language_English',
 'country_USA',
 'content_rating_PG-13',
 'budget_high',
 'title_year_high',
 'actor_2_facebook_likes_medium',
 'imdb_score_low',
 'aspect_ratio_medium',
 'movie_facebook_likes_high']

In [12]:
# Hyperparameters and training

num_features = 128
min_word_count = 1
num_workers = multiprocessing.cpu_count()
context_size = 10
downsampling = 1e-3
seed = 123

model = word2vec.Word2Vec(
    tokenized_sentences,
    sg=1, # for skip gram
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling,
    iter=10
)

In [ ]:
# Compress the model to 2D
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)
all_word_vectors_matrix = model.wv.syn0
all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix)

In [ ]:
colors = {'genre': 1, 'plot': 1, 'director':3}
points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[model.wv.vocab[word].index])
            for word in model.wv.vocab
        ]
    ],
    columns=["word", "x", "y"]
)

%pylab inline



In [ ]:
points['color'] = 0

In [ ]:
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")

sns.lmplot('x', 'y',
           data=points,
           fit_reg=False,
           scatter_kws={"marker": "D",
                        "s": 10})
plt.title('t-SNE')
plt.xlabel('X')
plt.ylabel('Y')

In [ ]:
def plot_region(x_bounds, y_bounds):
    slice = points[
        (x_bounds[0] <= points.x) &
        (points.x <= x_bounds[1]) & 
        (y_bounds[0] <= points.y) &
        (points.y <= y_bounds[1])
    ]
    
    ax = slice.plot.scatter("x", "y", s=35, figsize=(10, 8))
    for i, point in slice.iterrows():
        ax.text(point.x + 0.005, point.y + 0.005, point.word, fontsize=11)

In [ ]:
plot_region(x_bounds=(0.0, 0.2), y_bounds=(0.0, 0.2))

In [179]:
model.wv.

array([-0.13049535,  0.05881098,  0.12270202,  0.09838531,  0.13205636,
        0.06448915, -0.20486678,  0.00444005,  0.21556512, -0.0716982 ,
        0.00519379, -0.2883549 ,  0.11012977, -0.06077716,  0.07668479,
        0.26939711,  0.23944943, -0.16256697,  0.14142168,  0.11269884,
        0.11095925, -0.09057152, -0.11743756, -0.16584449,  0.07440011,
       -0.06001869,  0.2445102 ,  0.18840589,  0.34108984, -0.08446051,
        0.06635012,  0.02845452,  0.21246713,  0.14055717,  0.00928075,
        0.03632695, -0.17384207, -0.23710252, -0.12127564,  0.01456438,
        0.22097614,  0.00709927,  0.2507517 , -0.0772662 , -0.20985299,
        0.06874697,  0.12411286,  0.08478492,  0.03403108, -0.14745893,
       -0.23782755,  0.23149908, -0.19273543,  0.01799721,  0.20531186,
        0.1835835 , -0.00243864, -0.16327122,  0.18075036,  0.18875886,
       -0.40238693, -0.28551352,  0.18977487,  0.04106176, -0.0267708 ,
        0.25772169,  0.05636012, -0.09839606,  0.08687349,  0.06

In [13]:
def likeBradPitt(v):
    if not v:
        return False
    return str(v) != 'Brad_Pitt' and model.wv.similarity(v, 'Brad_Pitt') > 0.8

def likeToken(v, token):
    try:
        return model.wv.similarity(v, token) > 0.5
    except:
        return False

In [14]:
df[(df.movie_title.apply(lambda y: likeToken('movie_title_' + y, 'comedy'))) & (df.actor_1_name.apply(lambda x : likeBradPitt(x)))]

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
2545,Black and White,Michel_Hazanavicius,high,medium,high,high,Ed_Lauter,medium,medium,Comedy|Drama|Romance,Bérénice_Bejo,The_Artist,high,medium,Beth_Grant,low,1920s|jack russell terrier|modern silent movie...,high,English,France,PG-13,medium,high,high,high,low,high
2244,Color,Shana_Feste,medium,medium,medium,high,Bruce_Greenwood,high,medium,Drama|Romance,Alex_Pettyfer,Endless_Love,medium,high,Rhys_Wakefield,low,brother sister relationship|high school gradua...,low,English,USA,PG-13,medium,high,high,medium,medium,low
3104,Color,Dennis_Dugan,medium,low,high,medium,Kevin_Nealon,high,medium,Comedy|Sport,Adam_Sandler,Happy_Gilmore,high,high,Frances_Bay,low,golf|golf ball|golfer|grandmother's house|tour...,high,English,USA,PG-13,medium,low,medium,medium,low,low
4381,Color,Todd_Field,high,high,high,high,Sissy_Spacek,medium,medium,Crime|Drama,Tom_Wilkinson,In_the_Bedroom,medium,medium,Karen_Allen,low,doctor|maine|only son|school choir|summer,high,English,USA,R,low,low,high,high,medium,high
1952,Color,Paul_Thomas_Anderson,high,low,low,low,Emily_Watson,high,medium,Comedy|Drama|Romance|Thriller,Adam_Sandler,Punch-Drunk_Love,high,high,Don_McManus,low,harmonium|love|phone sex|pudding|sister,high,English,USA,R,medium,medium,high,high,medium,low
4287,Color,Marielle_Heller,high,medium,medium,low,David_Fine,high,low,Drama|Romance,Alexander_Skarsgård,The_Diary_of_a_Teenage_Girl,medium,high,Austin_Lyon,high,1970s|mother's boyfriend|overalls|phone booth|...,low,English,USA,R,low,high,high,medium,medium,low
3463,Color,Jason_Reitman,high,low,high,high,Jennifer_Garner,high,high,Comedy|Drama|Romance,J.K._Simmons,Juno,high,high,Rainn_Wilson,low,adoption|baby|friend|pregnancy|school,high,English,USA,PG-13,low,medium,high,high,low,high
2085,Color,Andy_Fickman,medium,medium,medium,high,Kyra_Sedgwick,high,high,Comedy|Family|Sport,Dwayne_Johnson,The_Game_Plan,medium,high,Madison_Pettis,low,family relationships|football movie|publicist|...,low,English,USA,PG,medium,medium,high,medium,medium,low
3926,Color,Jeff_Franklin,low,low,medium,high,Tyra_Banks,medium,low,Comedy,Ivana_Milicevic,Love_Stinks,low,medium,Bridgette_Wilson-Sampras,high,black comedy|marriage|marriage proposal|not ta...,low,English,USA,R,low,low,medium,low,low,high
747,Color,Brian_Robbins,medium,medium,medium,high,Joel_David_Moore,high,high,Comedy|Family|Fantasy,Robert_Downey_Jr.,The_Shaggy_Dog,low,high,Kristin_Davis,low,animal as human|dog movie|elevator|human becom...,low,English,USA,PG,high,medium,high,low,medium,high


In [ ]:
df.apply(la)